<a href="https://colab.research.google.com/github/anuthereaper/PythonLibrary/blob/main/Servicebus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/servicebus/azure-servicebus/samples/async_samples
https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/servicebus/azure-servicebus/samples/sync_samples
https://docs.microsoft.com/en-us/azure/service-bus-messaging/service-bus-python-how-to-use-queues

In [ ]:
pip install azure.servicebus

     |████████████████████████████████| 200 kB 4.2 MB/s 
     |████████████████████████████████| 85 kB 3.3 MB/s 
     |████████████████████████████████| 45 kB 3.3 MB/s 
     |████████████████████████████████| 176 kB 44.9 MB/s 
     |████████████████████████████████| 3.1 MB 45.0 MB/s 


In [ ]:
# Receive messages from service bus
from azure.servicebus import ServiceBusClient
import pandas as pd
import json
from pandas import json_normalize

CONNECTION_STR = 'Endpoint=sb://<servicebusnamespace>.servicebus.windows.net/;SharedAccessKeyName=<accesskeyname>;SharedAccessKey=<Access_Key>;EntityPath=<Service_Bus_Name>'
QUEUE_NAME = '<Service_Bus_Name>'

servicebus_client = ServiceBusClient.from_connection_string(conn_str=CONNECTION_STR)

with servicebus_client:
    receiver = servicebus_client.get_queue_receiver(queue_name=QUEUE_NAME)
    
    # create an Empty DataFrame object
    df = pd.DataFrame()
    msg_concat = ""
    dfs = []
    with receiver:
        received_msgs = receiver.receive_messages(max_message_count=10, max_wait_time=5)  # Receive a batch of 10 messages
        for msg in received_msgs:
            msg_dict = json.loads(str(msg))
            df2 = json_normalize(msg_dict)
            df = df.append(df2, ignore_index = True)
            receiver.complete_message(msg)
print(df)
print("Receive is done.")

  Name      Seri SiName As  ...  Id  Asse                                    id  t
0       21000000            ...   0  None  075f0a38-2816-42c7-b95c-66c425b8ba9d -1
1        4560000            ...   0  None  075f0a38-2816-42c7-b95c-66c425b8ba9d -2

[2 rows x 21 columns]
Receive is done.


In [ ]:
from azure.servicebus import ServiceBusMessage
# Send messages to service bus   https://docs.microsoft.com/en-us/azure/service-bus-messaging/service-bus-python-how-to-use-queues
from azure.servicebus import ServiceBusClient

CONNECTION_STR = 'Endpoint=sb://<servicebusnamespace>.servicebus.windows.net/;SharedAccessKeyName=<accesskeyname>;SharedAccessKey=<Access_Key>;EntityPath=<Service_Bus_Name>'
QUEUE_NAME = '<Service_Bus_Name>'

servicebus_client = ServiceBusClient.from_connection_string(conn_str=CONNECTION_STR)

def send_single_message(sender):
    # create a Service Bus message
    message = ServiceBusMessage("Single Message")
    # send the message to the queue
    sender.send_messages(message)
    print("Sent a single message")

def send_a_list_of_messages(sender):
    # create a list of messages
    messages = [ServiceBusMessage("Message in list") for _ in range(5)]
    # send the list of messages to the queue
    sender.send_messages(messages)
    print("Sent a list of 5 messages")

def send_batch_message(sender):
    # create a batch of messages
    batch_message = sender.create_message_batch()
    for _ in range(10):
        try:
            # add a message to the batch
            batch_message.add_message(ServiceBusMessage("{\"Name\":\"\",\"Seri\":21000000,\"SiName\":\"\",\"As\":\"\"}"))
        except ValueError:
            # ServiceBusMessageBatch object reaches max_size.
            # New ServiceBusMessageBatch object can be created here to send more data.
            break
    # send the batch of messages to the queue
    sender.send_messages(batch_message)
    print("Sent a batch of 10 messages")

with servicebus_client:
    sender = servicebus_client.get_queue_sender(queue_name=QUEUE_NAME)
    with sender:
        send_single_message(sender)
        send_a_list_of_messages(sender)
        send_batch_message(sender)